# Deploy a Serverless Model Server with Nuclio-KFServing
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy a **[LighGBM](https://github.com/microsoft/LightGBM)** model using **[nuclio](https://github.com/nuclio/nuclio)** + **[KFServing](https://github.com/kubeflow/kfserving)** (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving (KFServing) class in a notebook.
* Deploy the model server as a Nuclio-serving function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [28]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [29]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [30]:
%%nuclio cmd -c
pip install -U -q kfserving
pip install -U -q azure
pip install -U -q numpy
pip install -U -q xgboost
pip install -U -q lightgbm
pip install -U -q mlrun

In [31]:
import kfserving
import os
import numpy as np
from pickle import load
import lightgbm as lgb

**NOTE:   bring your own pickled model by changing the following variables, or run the [LightGBM demo](https://github.com/mlrun/demos/tree/master/lightgbm#instructions-for-lightgbm-demo).**

In [32]:
model = ()
TARGET_PATH = '/User/mlrun/lightgbm'
MODEL_FILE = 'lightgbm_classifier.pkl'

In [6]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, model = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not model is None:
            self.classifier = model
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_FILE)
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body):
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [7]:
# nuclio: end-code

______________________________________________

<a id='test-locally'></a>
### **test the model serving class locally**
The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a `model.bst` file in the model_dir path (generated by the training notebook)**

In [16]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

In [17]:
my_server = ClassifierModel('some-classifier-model', model_dir=TARGET_PATH)
my_server.load()

[I 200120 17:03:17 storage:35] Copying contents of /User/mlrun/lightgbm to local


### _data_
Grab some data from the test set we prepared in the **[LightGBM demo](https://github.com/mlrun/demos/tree/master/lightgbm#instructions-for-lightgbm-demo)**:

In [18]:
features = pq.read_table(os.path.join(TARGET_PATH, 'xtest.parquet')).to_pandas().iloc[:3, :]
labels = pq.read_table(os.path.join(TARGET_PATH, 'ytest.parquet')).to_pandas().iloc[:3, :]

In [19]:
event = {"instances": features.values.tolist()}


We can use the `.predict(body)` method to test the model.

In [20]:
my_server.predict(event)

[1.0, 1.0, 0.0]

<a id='deploy'></a>
### **deploy our serving class using as a serverless function**
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

the `models` dict store model names and the assosiated model **dir** URL (the URL can start with `S3://` and other blob store options), the faster way is to use a shared file volume, we use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `\User` function path.

**verify the model dir does contain a valid `model.bst` file**

In [21]:
from mlrun import new_model_server, mount_v3io
import requests

In [23]:
fn = new_model_server('some-classifier-model', 
                      models={'classifier_gen': TARGET_PATH}, 
                      model_class='ClassifierModel')

fn.apply(mount_v3io()) 

In [24]:
addr = fn.deploy()

[mlrun] 2020-01-20 17:05:07,328 deploy started
[nuclio] 2020-01-20 17:07:37,734 (info) Build complete
[nuclio] 2020-01-20 17:07:46,843 done creating some-classifier-model, function address: 3.135.246.153:31529


<a id="test-model-server"></a>
### **test our model server using HTTP request**


We invoke our model serving function using test data, the data vector is specified in the `instances` attribute.

In [25]:
import json
resp = requests.post(addr + '/classifier_gen/predict', json=event)

In [26]:
json.loads(resp.content)

[1.0, 1.0, 0.0]

**[back to top](#top)**